<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/Matrix_Factorization_Freq_Shoppers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Embeddings Using Matrix Factorization

This notebook provides an overview of developing embeddings for the retail example using $\textbf{Non-Negative Matrix Factorization}$. The perspective in this exercise is data analysis rather a specific machine learning task. The approach illustrated in this notebook provides both insights as well as embeddings. Developing the embeddings also requires fewer hyper-parameter choices than approaches such as $\textbf{node2vec}$. The data for this exercise is the feature selected set of customer purchases of the frequent shopper group developed in [notebook2](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/Graph_Retail_EDA_II.ipynb) of the retail data analysis with ArangoDB series. The details of the anaysis are provided below.

This note book produces a _knowledge graph_ that captures the preferences and shopping behavior of frequent shoppers. This _knowledge graph_ is stored in ArangoDB. This notebook develops _embeddings_ of the customers and store items. These embeddings are stored as _node properties_ of the _knowledge graph_ . The _embeddings_ can be used for similarity search and in _adhoc_ queries. Examples of performing this in _AQL_ are provided. 

In [ ]:
%%capture
get_ipython().__class__.__name__ = "ZMQInteractiveShell"
!git clone -b retail_data_branch --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av  interactive_tutorials/notebooks/data  rsync -av interactive_tutorials/notebooks/tools interactive_tutorials/notebooks/img ./ --exclude=.git
!pip install python-arango
!pip install arangopipe==0.0.70.0.0
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle
!pip3 install networkx
!pip3 install matplotlib
!pip3 install adbnx-adapter==0.0.0.2.5.3
!pip install hdbscan
!pip install seaborn
!chmod a+x tools/*

![](/content/img/retail_freq_customer_purchases.png)

## Read Feature Selected Frequent Shoppers
This dataset captures the feature selected frequent shoppers after feature selection using the LASSO (see [retail data series, exploratory data analysis](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/Graph_Retail_EDA_II.ipynb) for details).

In [ ]:
import pandas as pd
fp = "data/feature_selected_freq_shoppers.csv"
df = pd.read_csv(fp)
df.head()

## Bipartite Graph Representation
This feature selected set of purchases of the frequent customers can be viewed as a bi-partite graph with the customers and the store inventory items constituting the disjoint vertex sets.

![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/retail_freq_customer_purchases.png?raw=1)


## Overview of Embeddings from Matrix Factorization

An embedding of this graph will be developed using matrix factorization. The feature selected set of purchases, $\mathbf{P}$, is the matrix we want to factorize. This is factorized into a tall skinny matrix, $\mathbf{W}$, and a shallow wide matrix, $\mathbf{H}$, as shown below.

![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/matrix_factorization_algorithm_schematic.png?raw=1)

The store inventory of $m$ items is grouped into $k$ purchase groups. The purchases of the $i^{th}$ customer, a vector of size $m$, is expressed as a product of the purchase groups and the customers affinities towards the purchase groups as follows:


$$\boxed{p_i = u_{(i,k)}. pg_{(k, m)}}$$



The critical aspect of this factorization process is the determination of the latent dimension, $k$. Regularization can be factored into the matrix factorization process as discussed in the [sklearn non negative matrix factorization implementation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). However, since we have explicitly used the LASSO with the notion of RFM score to select the features, the regularization parameters are set to zero in the matrix factorization objective function. To determine the latent dimension, $k$, a test set of purchases is held out. A range of possible values for $k$, from $2$ to $20$, were used. The mean square error for each value of $k$ is recorded. The value of $k$ that produced the lowest MSE is selected as the best value of $k$. The flowchart for the process is shown below.

![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/matrix_factorization_flowchart.png?raw=1)





In [ ]:
cust_ids = df["Customer_ID"]
rfm_scores = df["RFM_score"]
to_del = ["Customer_ID", "RFM_score"]
cols = df.columns.tolist()
req_cols = [c for c in cols if c not in to_del]
df = df[req_cols]

### Hold out $\frac{1}{3}$ of the matrix of purchases as the test set.

In [ ]:
TEST_PROP = 0.33
NUM_TEST_CELLS = int(df.shape[0] * df.shape[1]* TEST_PROP)

In [ ]:
import random
import numpy as np
dict_test = dict()
num_test_samples = 0
random.seed(1234)
while num_test_samples <= NUM_TEST_CELLS:
    row = random.randint(0, (df.shape[0] -1))
    col = random.randint(0, (df.shape[1] - 1))
    dict_test[(row, col)] = df.iloc[row, col]
    df.iloc[row, col] = 0.0
    num_test_samples += 1
    

In [ ]:
X = df.values

In [ ]:
X.shape

### Try a range of factors, $2$ to $20$, as possible latent dimensions and compute the mean square error.


In [ ]:
from sklearn.decomposition import NMF

#np.random.seed(42)
exp_factors = np.arange(2,21)
recon_err = dict()
for fac in exp_factors:
    print("Computing factor %d" % (fac))
    model = NMF(n_components=fac, init='random', random_state=1234)
    W = model.fit_transform(X)
    H = model.components_
    se = []
    lrm = W.dot(H)
    for k, v in dict_test.items():
        sqerr = (lrm[k[0], k[1]] - v)**2
        se.append(sqerr)
    recon_err[fac] = np.sum(se)/NUM_TEST_CELLS

    

### Plot latent dimension versus mean square error.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [11, 8]
sns.set(color_codes=True)
keys = np.fromiter(recon_err.keys(), dtype=int)
vals = np.fromiter(recon_err.values(), dtype=float)
plt.xticks(range(1,20))
sns.lineplot(keys, vals, marker = 'o')

### Determine the optimal latent dimension

In [ ]:
OPTIMAL_EMBEDDING_SIZE = 5
n_top_words = 10
model = NMF(n_components= OPTIMAL_EMBEDDING_SIZE, init='random', random_state=1234)
W = model.fit_transform(X)
H = model.components_
topic_features = dict()

## Matrix Factorization
Once the optimal latent dimension, $k$, has been identified, we can compute the matrix factorization of the purchases, $\mathbf{P}$. This factorization provides the following approximation

$$\mathbf{P} \approx \mathbf{W}. \mathbf{H}$$

where:

$\mathbf{H}$: Represents the dictionary matrix. This can be interpretted as $k$ groups of store inventory items. Each of these groups is of size, $m$, the number of items in the store inventory ($75$). A single purchase group is a vector of size $m$. The magnitude of the vector elements provides an indication of the store inventory items that dominate a purchase group. As will be observed subsequently, each of these purchase groups are dominated by a small number of items. The magnitude of most of the elements of the purchase group vector are small. 

$\mathbf{W}$: Represents the affinity matrix. This can be interpretted as a matrix that captures the affinities customers have towards each of the $k$ purchase groups. 

The elements of $\mathbf{W}$ are the representation of the customers in the $k$ dimensional latent space. The elements of $\mathbf{H}$ are the representations of the items in the $k$ dimensional latent space. Therefore, Matrix Factorization provides the following:


1.   Vector representations of the customers and store inventory in a $k$ dimensional space.
2.   An interpretation of customer purchases in terms of dictionary learning. Store inventory can be grouped into $k$ purchase groups. The purchases of custormers can be viewed in terms of their affinities towards item groups.

Since $k$ is smaller than $m$, we are able to group the store inventory into a small number of groups and view customer preferences in terms of their affinities towards these groups. Therefore matrix factorization provides embeddings as well as insights into the tastes of the frequent shopper group. 

When embeddings for the frequent customer purchase group are developed using techniques such as node2vec, we have to specify a few more hyper-parameters to develop the embedding. For example, with node2vec, we have to specify the parameters of the random-walk, the number of walks and the length of the walk as well as the embedding size (the latent dimension $k$ in this example). Usually these are determined through a hyper-parameter optimization procedure, for example, Bayesian Optimization. In this example, we have only one hyper-parameter, the latent dimension, $k$ and this was determined through a principled experiment.




In [ ]:
fp = "data/stock_code_description.csv"
dfsclu = pd.read_csv(fp)
lusc = dict()
for index, row in dfsclu.iterrows():
    lusc[row['StockCode']] = row["Description"]
del dfsclu

## Dictionary Profile
The top $10$ components of each of the $5$ dictionary vectors are plotted

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    
    fig, axes = plt.subplots(1, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]
        topic_features[topic_idx] = top_features

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Item Group {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()
    

In [ ]:
plot_top_words(model, req_cols, n_top_words, "MF Based Embedding of Frequent Customers")

### Determine the description of the dominant components of the dictionary elements

In [ ]:
XH = H
dfH = pd.DataFrame(XH)
from sklearn import preprocessing

Xh = preprocessing.normalize(dfH.values)
dfHn = pd.DataFrame(Xh)
dfHn

In [ ]:
import seaborn as sns
from matplotlib import rcParams
fig, ax = plt.subplots(figsize=(15,6)) 
igl = ["Item Group_" + str(i) for i in range(6)]
sns.set(color_codes=True)
cmap = sns.light_palette((260, 75, 60), input="husl")
ax = sns.heatmap(dfHn, cmap = cmap)
l = ax.set_xticklabels(req_cols, rotation = 90)
l = ax.set_yticklabels(igl, rotation= 0)

In [ ]:
TOP_N = 3
iglist = []
itemlist = []
desclist = []
for ig in range(OPTIMAL_EMBEDDING_SIZE):
  dom_itms = np.argsort(H[ig])[:TOP_N]
  for itm in dom_itms:
    iglist.append("Item Group_" + str(ig))
    itemlist.append(itm)
    desclist.append(lusc[req_cols[itm]])


In [ ]:
df_dict_profile = pd.DataFrame(data = {"Item_group": iglist, "Item_ID": itemlist, "Description": desclist})
df_dict_profile

In [ ]:
for k,v in topic_features.items():
    item_desc = [lusc[i] for i in v]
    print("Item Group %d, elements:" %(k+1))
    for i, desc in enumerate(item_desc):
        print(desc)
        if i >=4:
            break
    print()
        
    

In [ ]:
W.shape

## Customer Profile

To develop the customer profile, the interpretation of the customer representation as his or her affinity towards the $k$ latent groups is utilized. The set $k$ dimensional customer representations are clustered using a density clustering algorithm $\textbf{hdbscan}$. This algorithm can discriminate between points that genuinely cluster together versus noise. Many real world datasets present with background noise. Removing this background noise helps us identify the actual clusters in the dataset. It turns out that this dataset has a large noise cluster.

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_samples=5)
clusterer.fit(W)

### There are $6$ clusters in data apart from the noise cluster

In [ ]:
np.unique(clusterer.labels_)

In [ ]:
clusterer.labels_.shape

In [ ]:
W.shape

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=1234)
Xt = model.fit_transform(X) 


In [ ]:
dft = pd.DataFrame(Xt)

In [ ]:
dft["cluster"] = clusterer.labels_

## Profile of the clusters
* There is a large noise cluster
* There is a large cluster (cluster label $4$) and $5$ smaller clusters. The cluster sizes are shown below.

In [ ]:
cluster_counts = dft["cluster"].value_counts()
cluster_counts.plot.barh()

In [ ]:
dfnf = dft.query("cluster != -1")
dfnf.columns = ["X", "Y", "cluster"]

In [ ]:
dfW = pd.DataFrame(W)
dfW.columns = ["Item Group_"+ str(i) for i in range(5)]
dfW["cluster"] = clusterer.labels_
dfW = dfW.query("cluster != -1")
dfWc = dfW.groupby(by = ["cluster"]).mean()

In [ ]:
colsneeded = ["Item Group_"+ str(i) for i in range(5)]
dfWc = dfWc[colsneeded]
dfWc

In [ ]:
from sklearn import preprocessing
Xc = preprocessing.normalize(dfWc.values)
dfWcn = pd.DataFrame(Xc)
dfWcn.columns = colsneeded
dfWcn

In [ ]:
fig, ax = plt.subplots(figsize=(15,9.5)) 
ccl = ["Cluster-" + str(i) for i in range(6)]
sns.set(color_codes=True)
cmap = sns.light_palette("purple")
ax = sns.heatmap(dfWcn, cmap = cmap)
l = ax.set_xticklabels(colsneeded, rotation = 90)
l = ax.set_yticklabels(ccl)

## Plot the Customer Clusters using tSNE

In [ ]:
plt.rcParams['figure.figsize'] = [11, 8]
sns.set(color_codes=True)
Xnf= dfnf["X"].values
Ynf = dfnf["Y"].values
sns.scatterplot(Xnf, Ynf, hue = dfnf["cluster"])

## Frequent Customer Graph
The graph that captures frequent store customers and their purchasing activity are represented as a graph. This graph and the embeddings of the customers as well as items are stored in ArangoDB. The details of doing so are provided below.

In [ ]:

from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https',\
                        msc.DB_REPLICATION_FACTOR: 3}
        
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = False, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)
proj_info = {"name": "Retail_Graph_Analytics"}
proj_reg = admin.register_project(proj_info)
mdb_config.get_cfg()

In [ ]:

ds_info = {"name" : "Frequent_Customer_Graph",\
            "description": "Frequent Customer Graph of Retail Dataset",\
           "source": "ArangoDB dump of Retail from Notebook II" }
ds_reg = ap.register_dataset(ds_info)

In [ ]:
featureset = {"name": "FC_NO_FS"}
fs_reg = ap.register_featureset(featureset, ds_reg["_key"])

In [ ]:
import io
import requests
url = ('https://raw.githubusercontent.com/arangodb/interactive_tutorials/master/notebooks/Matrix_Factorization_Freq_Shoppers.ipynb')
nbjson = requests.get(url).text

In [ ]:
model_info = {"name": "MF for Frequent Customers",  "task": "Develop Embeddings", 'notebook': nbjson}
model_reg = ap.register_model(model_info, project = "Retail_Graph_Analytics")

In [ ]:
import uuid
import datetime
#import jsonpickle

ruuid = str(uuid.uuid4().int)
model_perf = {'run_id': ruuid, "timestamp": str(datetime.datetime.now())}

#mp = jsonpickle.encode(mp)
model_params = {'run_id': ruuid, 'model_params': 'Not Applicable'}

run_info = {"dataset" : ds_reg["_key"],\
                    "featureset": fs_reg["_key"],\
                    "run_id": ruuid,\
                    "model": model_reg["_key"],\
                    "model-params": model_params,\
                    "model-perf": model_perf,\
                    "tag": "Retail Descriptive Analytics Notebook",\
                    "project": "Retail_Graph_Analytics"}
ap.log_run(run_info)

In [ ]:
kg_db = admin.db

## Create the collections to store the frequent customer graph

In [ ]:

# create a new graph called freq_cust_graph in the temp database if it does not already exist.
if not kg_db.has_graph("freq_cust_graph"):
    kg_db.create_graph('freq_cust_graph', smart=True)

# # create a new graph called user_user_graph in the temp database if it does not already exist.
# if not kg_db.has_graph("user_user_graph"):
#     kg_db.create_graph("user_user_graph", smart=True)

# # This returns and API wrapper for the above created graphs
freq_cust_graph = kg_db.graph("freq_cust_graph")
# user_user_graph = kg_db.graph("user_user_graph")

# create a new collection named "Customers" if it does not exist.
# This returns an API wrapper for "Customers" collection.
if not kg_db.has_collection("Customers"):
    kg_db.create_collection("Customers", replication_factor=3)

# Create a new vertex collection named "Customers" if it does not exist.
if not freq_cust_graph.has_vertex_collection("Customers"):
    freq_cust_graph.vertex_collection("Customers")

# create a new collection named "Items" if it does not exist.
# This returns an API wrapper for "Items" collection.
if not kg_db.has_collection("Items"):
    kg_db.create_collection("Items", replication_factor=3)

# Create a new vertex collection named "Items" if it does not exist.
if not freq_cust_graph.has_vertex_collection("Items"):
    freq_cust_graph.vertex_collection("Items")

# create a new collection named "Purchases" if it does not exist.
# This returns an API wrapper for "Purchases" collection.
if not kg_db.has_collection("Purchases"):
    kg_db.create_collection("Purchases", edge=True, replication_factor=3)

# creating edge definitions named "Purchases". This creates any missing
# collections and returns an API wrapper for "Purchases" edge collection.
if not freq_cust_graph.has_edge_definition("Purchases"):
    Purchases = freq_cust_graph.create_edge_definition(
        edge_collection='Purchases',
        from_vertex_collections=['Customers'],
        to_vertex_collections=['Items']
    )


In [ ]:
df.shape[0]

### Store the customers and their embeddings

In [ ]:
import json
%time
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')
doc_index = 0

batch = []
BATCH_SIZE = 100
batch_idx = 1
collection =kg_db["Customers"]
cust_cluster = clusterer.labels_

for count, value in enumerate(cust_ids):
  insert_doc = {}
  user_id = "Customers/" + str(value)
  insert_doc["_id"] = user_id
  insert_doc["mf_emb"] = np.asarray(W[count], dtype = np.float32).tolist()
  insert_doc["RFM_score"] = str(rfm_scores[count])
  insert_doc["cluster"] = str(cust_cluster[count])
  batch.append(insert_doc)
  doc_index += 1
  last_record = (count == (df.shape[0] - 1)) 
  if doc_index % BATCH_SIZE == 0:
      print("Inserting batch %d" % (batch_idx))
      batch_idx += 1
      collection.import_bulk(batch)
      batch = []
  if last_record and len(batch) > 0:
      print("Inserting batch the last batch!")
      collection.import_bulk(batch)

### Stoe the store inventory items and their embeddings

In [ ]:
import json
%time

doc_index = 0

batch = []
BATCH_SIZE = 30
batch_idx = 1
collection =kg_db["Items"]
Ht = H.T
for count, value in enumerate(req_cols):
  insert_doc = {}
  item_id = "Items/" + str(value)
  insert_doc["_id"] = item_id
  insert_doc["mf_emb"] = np.asarray(Ht[count], dtype = np.float32).tolist()
  insert_doc["desc"] = lusc[value]
  batch.append(insert_doc)
  doc_index += 1
  last_record = (count == (len(req_cols) - 1)) 
  if doc_index % BATCH_SIZE == 0:
      print("Inserting batch %d" % (batch_idx))
      batch_idx += 1
      collection.import_bulk(batch)
      batch = []
  if last_record and len(batch) > 0:
      print("Inserting batch the last batch!")
      collection.import_bulk(batch)

### Store purchasing activity (captured as an edge)

In [ ]:
import json
%time

doc_index = 0

batch = []
BATCH_SIZE = 500
batch_idx = 1
collection =kg_db["Purchases"]
# returns all the edges from t10cu nodes to the nodes which are at 1-hop distance 

for index, row in df.iterrows():
  for col in req_cols:
    if row[col] > 0:
      user_id = "Customers/" + str(cust_ids[index])
      item_id = "Items/" + str(col)
      insert_doc = {"_from": user_id, "_to": item_id}
      batch.append(insert_doc)
      doc_index += 1
  last_record = (index == (df.shape[0] - 1)) 
  if doc_index % BATCH_SIZE == 0:
      print("Inserting batch %d" % (batch_idx))
      batch_idx += 1
      collection.import_bulk(batch)
      batch = []
  if last_record and len(batch) > 0:
      print("Inserting batch the last batch!")
      collection.import_bulk(batch)

## Query to find customers similar to a customer

### AQL Query (can be run through the web UI queries tab)
The following query can be used to find customers similar to a customer
```/*locate specific user*/
LET uemb = (
FOR u in Customers
    FILTER u._id == "Customers/12748"
    FOR j in RANGE(0,4)
        RETURN TO_NUMBER(NTH(u.mf_emb,j))
    )

/*calculate distance from user to all other users*/
LET dau = (
    FOR v in Customers
    /* Limit to users that have an embedding*/
    FILTER HAS(v, "mf_emb")
    LET dv = (SQRT(SUM(
    
        FOR i in RANGE(0,4)
            LET di = TO_NUMBER(NTH(uemb, i)) - TO_NUMBER(NTH(v.mf_emb, i))
            RETURN POW(di,2)
        )))
    RETURN {"user": v._id, "dist": dv}
    )
/*sort results*/    
FOR du in dau
    SORT du.dist
    RETURN {"user": du.user, "dist": du.dist}
```


![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/mf_cust_similarity_query.png?raw=1)

## Query to find items similar to an item

### AQL Query (can be run through the web UI queries tab)
The following query can be used to find items  similar to an item

```
LET uemb = (
FOR u in Items
    FILTER u._id == "Items/22045"
    FOR j in RANGE(0,4)
        RETURN TO_NUMBER(NTH(u.mf_emb,j))
    )

/*calculate distance from item to all other items*/
LET dau = (
    FOR v in Items
    /* Limit to items that have an embedding*/
    FILTER HAS(v, "mf_emb")
    LET dv = (SQRT(SUM(
    
        FOR i in RANGE(0,4)
            LET di = TO_NUMBER(NTH(uemb, i)) - TO_NUMBER(NTH(v.mf_emb, i))
            RETURN POW(di,2)
        )))
    RETURN {"item": v._id, "dist": dv}
    )
```

![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/mf_item_similarity_query.png?raw=1)

## Query to find customers in a cluster

### AQL Query (can be run through the web UI queries tab)
The following query can be used to find customers in a cluster

```
for c in Customers
    FILTER c.cluster == '4'
        RETURN c 

```

![](https://github.com/arangodb/interactive_tutorials/blob/retail_data_branch/notebooks/img/mf_emb_cluster4_cust.png?raw=1)